In [ ]:
'''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

In [1]:
from IPython.display import clear_output
!pip install  transformers --quiet
!pip install rouge_score -q
!pip install deep-phonemizer -q
!pip install opendatasets -q
!pip install  datasets -q
!pip uninstall -y transformers accelerate -q
!pip install transformers accelerate -q
!pip install evaluate -q
!pip install pycocoevalcap -q
!pip install huggingface_hub -q
!pip install -U nltk 
clear_output()

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
!unzip   /usr/share/nltk_data/corpora/wordnet.zip -d   /usr/share/nltk_data/corpora/



Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [8]:
import requests

import os

import datasets
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
#from tqdm import tqdm
#from tqdm.auto import tqdm
#import multiprocessing as mp
import nltk
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import io, transforms
from torch.utils.data import Dataset, DataLoader, random_split

from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
from transformers import VisionEncoderDecoderModel , ViTImageProcessor, ViTFeatureExtractor
from transformers import GPT2Config , default_data_collator
from transformers import GPT2TokenizerFast
from transformers import pipeline,VisionEncoderDecoderConfig

import evaluate

if torch.cuda.is_available():    

    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


In [11]:
'''
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice '''

'\nfrom pycocoevalcap.cider.cider import Cider\nfrom pycocoevalcap.spice.spice import Spice '

In [12]:
#Loading data in colab
'''import opendatasets as od
od.download(
    "https://www.kaggle.com/datasets/adityajn105/flickr8k")'''

'import opendatasets as od\nod.download(\n    "https://www.kaggle.com/datasets/adityajn105/flickr8k")'

# Initializing the Encoder-Decoder Model

In [9]:
encoder_model = "microsoft/swin-base-patch4-window7-224-in22k"
decoder_model = "aubmindlab/aragpt2-base"

In [10]:
os.environ["WANDB_DISABLED"] = "true"

In [11]:
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_model, decoder_model).to(device)

Some weights of the model checkpoint at microsoft/swin-base-patch4-window7-224-in22k were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at aubmindlab/aragpt2-base and are newly initialized: ['h.0.crossattention.bias', 'h.4.crossattention.c_proj.bias', 'h.3.crossattention.masked_bias', 'h.2.ln_cross_attn.weight', 'h.10.crossattention.q_attn.weight', 'h.2.crossattention.c_proj.weight', 'h.11.crossattention.c_proj.weight', 'h.3.crossattention.q_attn.weight', 'h.10.crossattention.c_proj.weight', 'h.11.crossattention.c_attn.weight', 'h.2.crossattention.bias', 'h.1.crossattention.c_proj.bias', 'h.6.crossattention.c_proj.weight', 'h.10.ln_cross_attn.weight', 'h.11.crossattention.bias', 'h.7.crossattention.c_attn.weight', 'h.9.crossattention.masked_bias', 'h.3.crossattention.c_proj.weight', 'h.0.ln_cross_attn.weight', 'h.8.crossattention.masked_bias', 'h.1.crossattention.bias', 'h.4.crossattention.c_proj.weight', 'h.11.crossattention.q_attn.weight', 'h.7.ln_cross_attn.weight', 'h.7.crossattention.masked_bias', 'h.5.crossattention.c_proj.weight', 'h.

In [ ]:
#model.config

## Defining the Tokenizing and Feature Extractor

In [12]:
tokenizer = GPT2TokenizerFast.from_pretrained(decoder_model)
image_processor =  ViTFeatureExtractor.from_pretrained(encoder_model)

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [13]:
# Modifying some of the config params
tokenizer.pad_token = tokenizer.eos_token 
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5
model.config.num_return_sequences = 2
model.config.vocab_size = model.config.decoder.vocab_size


# Preparing the Dataset

In [22]:
#transformimg the images and converting them to tensors
train_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)), 
        transforms.ToTensor(),
        transforms.Normalize(
            mean=0, 
            std=1
        )
   ]
)


In [15]:
def load_document(filename):
    file=open(filename,'r',encoding='utf-8')
    text=file.read()
    file.close()
    return text

In [16]:
#Create dataframe that shows all captions:
#colab
#text = load_document('Flickr8k.arabic.txt')
#kaggle
text = load_document('/kaggle/input/flickr8k-arabic-captions/Flickr8k.arabic.txt')

In [17]:

datatxt = []
for line in text.split('\n'):
    col = line.split('\t')
    if len(col) == 1:
        continue
    w = col[0].split("#")
    datatxt.append(w + [col[1].lower()])

data = pd.DataFrame(datatxt,columns=["image","index","caption"])
data = data.reindex(columns =['index','image','caption'])
data = data[data.image != '2258277193_586949ec62.jpg']

uni_filenames = np.unique(data.image.values)

data.head()

,index,image,caption
0,0,1000268201_693b08cb0e.jpg,طفلة صغيرة تتسلق إلى مسرح خشبي
1,1,1000268201_693b08cb0e.jpg,طفلة صغيرة تتسلق الدرج إلى منزلها
2,2,1000268201_693b08cb0e.jpg,فتاة صغيرة في ثوب وردي تذهب إلى المقصورة الخشبية
3,0,1001773457_577c3a7d70.jpg,كلب أسود وكلب ثلاثي الألوان يلعبان مع بعضهما ا...
4,1,1001773457_577c3a7d70.jpg,كلب أسود وكلب أبيض ببقع بنية يحدقان في بعضهما ...


In [18]:
#train_df , val_df = train_test_split(data , test_size = 0.2)

split_ratio = 0.7
split_index = int(len(data) * split_ratio)

train_df  = data[:split_index]
val_df = data[split_index:]

split_ratio2 = 0.5
split_index2 = int(len(val_df) * split_ratio)

validation_df = val_df[:split_index2]
test_df= val_df[split_index2:]

In [19]:
class ImgDataset(Dataset):
    def __init__(self, df,root_dir,tokenizer,feature_extractor, transform = None):
        self.df = df
        self.transform = transform
        self.root_dir = root_dir
        self.tokenizer= tokenizer
        self.feature_extractor = feature_extractor
        self.max_length = 32
    def __len__(self,):
        return len(self.df)
    def __getitem__(self,idx):
        caption = self.df.caption.iloc[idx]
        image = self.df.image.iloc[idx]
        img_path = os.path.join(self.root_dir , image)
        img = Image.open(img_path).convert("RGB")
        

        if self.transform is not None:
            img= self.transform(img)
        pixel_values = self.feature_extractor(img, return_tensors="pt").pixel_values.to(device)
        captions = tokenizer(caption, max_length=self.max_length, 
                             padding="max_length", truncation=True).input_ids
        
        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(captions)}
        return  encoding #{'pixel_values': pixel_values, 'labels':}

In [23]:
#Colab
'''train_dataset = ImgDataset(train_df, root_dir = "flickr8k/Images",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
validation_dataset = ImgDataset(validation_df, root_dir = "flickr8k/Images",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
test_dataset = ImgDataset(test_df, root_dir = "flickr8k/Images",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
'''
#Kaggle
train_dataset = ImgDataset(train_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
validation_dataset = ImgDataset(validation_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
test_dataset = ImgDataset(test_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)


In [24]:
for item in train_dataset:
    print("train_dataset", item["labels"].shape)
    print("train_dataset", item["pixel_values"].shape)
    break
for item in validation_dataset:
    print("validation_dataset", item["labels"].shape)
    print("validation_dataset", item["pixel_values"].shape)
    break
for item in test_dataset:
    print("test_dataset", item["labels"].shape)
    print("test_dataset", item["pixel_values"].shape)
    break

train_dataset torch.Size([32])
train_dataset torch.Size([3, 224, 224])
validation_dataset torch.Size([32])
validation_dataset torch.Size([3, 224, 224])
test_dataset torch.Size([32])
test_dataset torch.Size([3, 224, 224])


# Training the Model

## Evaluation Metrics

In [40]:

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
meteor =  evaluate.load("meteor")

def compute_metrics(eval_pred):
    preds = eval_pred.label_ids
    labels = eval_pred.predictions

    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels_str = tokenizer.batch_decode(labels, skip_special_tokens=True)

    #rouge score
    rouge_result = rouge.compute(predictions=pred_str, references=labels_str)
    rouge_result = {k: round(v * 100, 4) for k, v in rouge_result.items()}#{k: round(v , 4) for k, v in rouge_result.items()}# 

    # meteor
    meteor_result = meteor.compute(predictions=pred_str, references=labels_str)

    # bleu scores
    bleu1_result = bleu.compute(predictions=pred_str, references=labels_str, max_order =1)
    bleu2_result = bleu.compute(predictions=pred_str, references=labels_str, max_order =2)
    bleu3_result = bleu.compute(predictions=pred_str, references=labels_str, max_order =3)
    bleu4_result = bleu.compute(predictions=pred_str, references=labels_str, max_order =4)
    
    generation_length = bleu4_result["translation_length"]
    '''#  cider
    # convert lists to dictionaries
    ref_captions = {image_id: [caption] for image_id, caption in enumerate(labels_str)}
    pred_captions = {image_id: [caption] for image_id, caption in enumerate(pred_str)}

    cider_scorer = Cider()
    cider_score, cider_scores = cider_scorer.compute_score(ref_captions, pred_captions)
    #spice
    spice_scorer = Spice()
    spice_score, spice_scores = spice_scorer.compute_score(ref_captions, pred_captions)'''
    return {
      **rouge_result, 
      "meteor": round(meteor_result['meteor']*100, 2) , #round(meteor_result['meteor'], 4),
      "bleu1": round(bleu1_result["bleu"] * 100, 4), #round(bleu1_result["bleu"] , 4), 
      "bleu2": round(bleu2_result["bleu"] * 100, 4), # round(bleu2_result["bleu"], 4), 
      "bleu3": round(bleu3_result["bleu"] * 100, 4), #round(bleu3_result["bleu"] , 4), 
      "bleu4": round(bleu4_result["bleu"]* 100 , 4),#round(bleu4_result["bleu"] , 4)
      #"cider": cider_score,
      #"spice": spice_scores
       #"gen_len": bleu4_result["translation_length"] / len(preds)
      }

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [41]:
# collate lists of samples into batches
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.stack([x['labels'] for x in batch])
    }

In [42]:
num_epochs = 10
batch_size = 16 

In [43]:

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,             
    num_train_epochs=num_epochs,      
    evaluation_strategy="steps", 
    learning_rate = 1.5e-6,
    optim = "adamw_torch",
    eval_steps=2500,                     
    logging_steps=2500,                   
    save_steps=2500,                        
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,  
    output_dir="AsmaMassad/swin-aragpt2-image-captioning-flickr8k",
    load_best_model_at_end=True,
    push_to_hub=True # push the model to huggingface hub,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [44]:
trainer = Seq2SeqTrainer(
    model=model,                  
    tokenizer=image_processor,    
    args=training_args,              
    compute_metrics=compute_metrics, 
    train_dataset=train_dataset,     
    eval_dataset=validation_dataset,      
    data_collator=collate_fn,        
)

/kaggle/working/AsmaMassad/swin-aragpt2-image-captioning-flickr8k is already a clone of https://huggingface.co/AsmaMassad/swin-aragpt2-image-captioning-flickr8k. Make sure you pull the latest changes with `repo.git_pull()`.


In [45]:
from torch.utils.data import DataLoader

def get_eval_loader(eval_dataset=None):
    return DataLoader(validation_dataset, collate_fn=collate_fn, batch_size=batch_size)

def get_test_loader(eval_dataset=None):
    return DataLoader(test_dataset, collate_fn=collate_fn, batch_size=batch_size)

trainer.get_train_dataloader = lambda: DataLoader(train_dataset, collate_fn=collate_fn, batch_size=batch_size)
trainer.get_eval_dataloader = get_eval_loader
trainer.get_test_dataloader = get_test_loader

In [46]:
trainer.train( )

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Meteor,Bleu1,Bleu2,Bleu3,Bleu4
2500,1.463700,1.288443,0.000000,0.000000,0.000000,0.000000,5.960000,9.530600,3.425800,1.499400,0.584400
5000,1.320100,1.230201,0.000000,0.000000,0.000000,0.000000,5.330000,8.672300,3.194400,1.432900,0.537700
7500,1.261700,1.213090,0.000000,0.000000,0.000000,0.000000,5.680000,9.265000,3.336300,1.409900,0.519300
10000,1.228000,1.208968,0.000000,0.000000,0.000000,0.000000,5.260000,8.178800,2.877200,1.234000,0.504900


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=10620, training_loss=1.3162173628582567, metrics={'train_runtime': 12097.4636, 'train_samples_per_second': 14.045, 'train_steps_per_second': 0.878, 'total_flos': 3.0837741499339407e+19, 'train_loss': 1.3162173628582567, 'epoch': 10.0})

In [47]:
# evaluate on the test_dataset
trainer.evaluate(test_dataset)

{'eval_loss': 1.2089682817459106,
 'eval_rouge1': 0.0,
 'eval_rouge2': 0.0,
 'eval_rougeL': 0.0,
 'eval_rougeLsum': 0.0,
 'eval_meteor': 5.26,
 'eval_bleu1': 8.1788,
 'eval_bleu2': 2.8772,
 'eval_bleu3': 1.234,
 'eval_bleu4': 0.5049,
 'eval_runtime': 423.3508,
 'eval_samples_per_second': 12.04,
 'eval_steps_per_second': 0.378,
 'epoch': 10.0}

In [ ]:
'''beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)'''